In [1]:
import re

In [17]:
chat2 = '''Abhishek R.
Hello Hussain, thank you for contacting GreenGeeks.
Hussain
site still down for two days baitalasaad.com
Customer ID: 173020
Support PIN: 605233
Abhishek R.
Thank you, please allow me a moment to get your identity verified first.
Copilot

I tagged this chat as "open/close" based on the canned response you sent
Internal note
Abhishek R.
I was able to successfully verify your account.
Just a moment while I check that for you. Thank you for your patience.
I can see that the website is working absolutely fine at my end, please check the screenshot: https://paste.websitehostserver.net/2025/07/08/text_QSUGwl4ZYP
Hussain
not working from my end last two days
and tired from different locations and browers
Abhishek R.
Could you please share a screenshot of what you are seeing on your end?
If you have a screenshot you wish to provide us, you can paste/upload your screenshot at https://snipboard.io/ or https://prnt.sc/

Once you have generated your screenshot, please provide me with the image URL so we can further assist you.
Copilot

I tagged this chat as "screenshot" based on the canned response you sent
Internal note
Hussain
https://snipboard.io/PTqsKx.jpg
Abhishek R.
Thank you, let me check on it. Please stay connected.   
Could you please try to access the following website on a different network than the current one, like a mobile hotspot, and check if it still gives the error?
Hussain
i tired everything
Abhishek R.
Please do as suggested below:
1. Clear Browser cookies/cache. You can refer to: http://www.wikihow.com/Clear-Your-Browser%27s-Cache

2. Clear DNS cache. Refer to: http://www.tech-faq.com/how-to-flush-dns.html

3. Reboot your Router.

4. Reboot your PC and try again to access your admin.

please make sure to follow each and every step if you still get the error please let us know.
Archived - customer left the chat
• 02:22 pm
'''

print(chat2)

Abhishek R.
Hello Hussain, thank you for contacting GreenGeeks.
Hussain
site still down for two days baitalasaad.com
Customer ID: 173020
Support PIN: 605233
Abhishek R.
Thank you, please allow me a moment to get your identity verified first.
Copilot

I tagged this chat as "open/close" based on the canned response you sent
Internal note
Abhishek R.
I was able to successfully verify your account.
Just a moment while I check that for you. Thank you for your patience.
I can see that the website is working absolutely fine at my end, please check the screenshot: https://paste.websitehostserver.net/2025/07/08/text_QSUGwl4ZYP
Hussain
not working from my end last two days
and tired from different locations and browers
Abhishek R.
Could you please share a screenshot of what you are seeing on your end?
If you have a screenshot you wish to provide us, you can paste/upload your screenshot at https://snipboard.io/ or https://prnt.sc/

Once you have generated your screenshot, please provide me with t

In [18]:
patterns = [
    r'\d{2}:\d{2}', # datestamp
    r'Copilot', #copilot
    r'canned', #canned
    r'Internal note', #internal notes
    r'Archived - customer left' # Archived message
]

In [19]:
def cleanup(chat_text, patterns):

    chat_by_line = chat_text.split('\n')
    return [
        line for line in chat_by_line
        if not any(re.search(p, line) for p in patterns)
    ]

In [20]:
cleaned = cleanup(chat_text=chat2, patterns = patterns)

In [21]:
cleaned

['Abhishek R.',
 'Hello Hussain, thank you for contacting GreenGeeks.',
 'Hussain',
 'site still down for two days baitalasaad.com',
 'Customer ID: 173020',
 'Support PIN: 605233',
 'Abhishek R.',
 'Thank you, please allow me a moment to get your identity verified first.',
 '',
 'Abhishek R.',
 'I was able to successfully verify your account.',
 'Just a moment while I check that for you. Thank you for your patience.',
 'I can see that the website is working absolutely fine at my end, please check the screenshot: https://paste.websitehostserver.net/2025/07/08/text_QSUGwl4ZYP',
 'Hussain',
 'not working from my end last two days',
 'and tired from different locations and browers',
 'Abhishek R.',
 'Could you please share a screenshot of what you are seeing on your end?',
 'If you have a screenshot you wish to provide us, you can paste/upload your screenshot at https://snipboard.io/ or https://prnt.sc/',
 '',
 'Once you have generated your screenshot, please provide me with the image URL 

#### Clean blanks, extra whitespace

In [22]:
cleaned = [line.strip() for line in cleaned if line != '']

In [23]:
cleaned

['Abhishek R.',
 'Hello Hussain, thank you for contacting GreenGeeks.',
 'Hussain',
 'site still down for two days baitalasaad.com',
 'Customer ID: 173020',
 'Support PIN: 605233',
 'Abhishek R.',
 'Thank you, please allow me a moment to get your identity verified first.',
 'Abhishek R.',
 'I was able to successfully verify your account.',
 'Just a moment while I check that for you. Thank you for your patience.',
 'I can see that the website is working absolutely fine at my end, please check the screenshot: https://paste.websitehostserver.net/2025/07/08/text_QSUGwl4ZYP',
 'Hussain',
 'not working from my end last two days',
 'and tired from different locations and browers',
 'Abhishek R.',
 'Could you please share a screenshot of what you are seeing on your end?',
 'If you have a screenshot you wish to provide us, you can paste/upload your screenshot at https://snipboard.io/ or https://prnt.sc/',
 'Once you have generated your screenshot, please provide me with the image URL so we can 

#### Split into speaker-message JSON format
Structure cleaned input into list of {speaker, message} blocks. Used for LLM input.

In [24]:
import re
from typing import List, Dict

def parse_chat(chat_lines: List[str]) -> List[Dict[str, str]]:
    agent = None
    customer = None
    speakers = []
    structured_chat = []

    # Step 1: Identify the agent by looking for canned response
    for i, line in enumerate(chat_lines):
        if "thank you for contacting" in line.lower():
            if i > 0:
                agent = chat_lines[i - 1].strip()
                break

    if not agent:
        raise ValueError("Agent could not be identified from canned message.")

    # Step 2: Identify customer as first different speaker
    for line in chat_lines:
        if re.match(r'^[A-Za-z0-9.@_\-\s]{1,30}$', line.strip()):  # likely a name/email/ID line
            name = line.strip()
            if name != agent:
                customer = name
                break

    if not customer:
        raise ValueError("Customer could not be identified.")

    # Step 3: Replace names and build role-labeled structure
    role_map = {agent: "agent", customer: "customer"}

    current_speaker = None
    message_buffer = []

    for line in chat_lines:
        line = line.strip()
        
        # Check if it's a speaker line
        if line in role_map:
            # Flush any buffered message
            if current_speaker and message_buffer:
                structured_chat.append({
                    current_speaker: " ".join(message_buffer).strip()
                })
                message_buffer = []
            current_speaker = role_map[line]
        else:
            message_buffer.append(line)

    # Add final message
    if current_speaker and message_buffer:
        structured_chat.append({
            current_speaker: " ".join(message_buffer).strip()
        })

    return structured_chat


In [25]:
parsed = parse_chat(cleaned)

In [26]:
parsed

[{'agent': 'Hello Hussain, thank you for contacting GreenGeeks.'},
 {'customer': 'site still down for two days baitalasaad.com Customer ID: 173020 Support PIN: 605233'},
 {'agent': 'Thank you, please allow me a moment to get your identity verified first.'},
 {'agent': 'I was able to successfully verify your account. Just a moment while I check that for you. Thank you for your patience. I can see that the website is working absolutely fine at my end, please check the screenshot: https://paste.websitehostserver.net/2025/07/08/text_QSUGwl4ZYP'},
 {'customer': 'not working from my end last two days and tired from different locations and browers'},
 {'agent': 'Could you please share a screenshot of what you are seeing on your end? If you have a screenshot you wish to provide us, you can paste/upload your screenshot at https://snipboard.io/ or https://prnt.sc/ Once you have generated your screenshot, please provide me with the image URL so we can further assist you.'},
 {'customer': 'https:/

In [ ]:
# testing local import 

ModuleNotFoundError: No module named 'src'